In [2]:
import csv
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load the preprocessed data
#the email matrix that has '1' for a presentation of a word, '0' for not.
email_data = np.load('email_data.npz')
email_data = email_data['arr_0']
#the label of emails
email_class = np.load('email_class.npy')

#print this two to see the words that are contianed in spam email and ham email
#all the spam words in a list
spam_word = open("spamword_list.txt","r").readlines()
#all the ham words in a list
ham_word = open("hamword_list.txt", "r").readlines()

#visuallize the original data
data = pd.read_csv("spam_ham.csv",encoding='latin-1')
data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":"label", "v2":"text"})
data['spam:-1_ham:1'] = data['label'].map( {'spam': -1, 'ham': 1} ).astype(int)
data['length'] = data['text'].apply(len)
data_ham  = data[data['spam:-1_ham:1'] == 1].copy()
data_spam = data[data['spam:-1_ham:1'] == -1].copy()
#uncomment to see content
print(data)

     label                                               text  spam:-1_ham:1  \
0      ham  Go until jurong point, crazy.. Available only ...              1   
1      ham                      Ok lar... Joking wif u oni...              1   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...             -1   
3      ham  U dun say so early hor... U c already then say...              1   
4      ham  Nah I don't think he goes to usf, he lives aro...              1   
5     spam  FreeMsg Hey there darling it's been 3 week's n...             -1   
6      ham  Even my brother is not like to speak with me. ...              1   
7      ham  As per your request 'Melle Melle (Oru Minnamin...              1   
8     spam  WINNER!! As a valued network customer you have...             -1   
9     spam  Had your mobile 11 months or more? U R entitle...             -1   
10     ham  I'm gonna be home soon and i don't want to tal...              1   
11    spam  SIX chances to win CASH! Fro

In [3]:
print(spam_word)

['free, entry, 2, wkly, comp, win, fa, cup, final, tkts, 21st, may, 2005, text, fa, 87121, receive, entry, questionstd, txt, ratetcs, apply, 08452810075over18s, freemsg, hey, darling, 3, weeks, word, back, id, like, fun, still, tb, ok, xxx, std, chgs, send, å£150, rcv, winner, valued, network, customer, selected, receivea, å£900, prize, reward, claim, call, 09061701461, claim, code, kl341, valid, 12, hours, mobile, 11, months, u, r, entitled, update, latest, colour, mobiles, camera, free, call, mobile, update, co, free, 08002986030, six, chances, win, cash, 100, 20000, pounds, txt, csh11, send, 87575, cost, 150pday, 6days, 16, tsandcs, apply, reply, hl, 4, info, urgent, 1, week, free, membership, å£100000, prize, jackpot, txt, word, claim, 81010, tc, wwwdbuknet, lccltd, pobox, 4403ldnw1a7rw18, xxxmobilemovieclub, use, credit, click, wap, link, next, txt, message, click, httpwap, xxxmobilemovieclubcomnqjkgighjjgcbl, england, v, macedonia, dont, miss, goalsteam, news, txt, ur, national, 

In [4]:
print(ham_word)

['go, jurong, point, crazy, available, bugis, n, great, world, la, e, buffet, cine, got, amore, wat, ok, lar, joking, wif, u, oni, u, dun, say, early, hor, u, c, already, say, nah, dont, think, goes, usf, lives, around, though, even, brother, like, speak, treat, like, aids, patent, per, request, melle, melle, oru, minnaminunginte, nurungu, vettam, set, callertune, callers, press, 9, copy, friends, callertune, im, gonna, home, soon, dont, want, talk, stuff, anymore, tonight, k, ive, cried, enough, today, ive, searching, right, words, thank, breather, promise, wont, take, help, granted, fulfil, promise, wonderful, blessing, times, date, sunday, oh, kim, watching, eh, u, remember, 2, spell, name, yes, v, naughty, make, v, wet, fine, thatåõs, way, u, feel, thatåõs, way, gota, b, seriously, spell, name, i\x89û÷m, going, try, 2, months, ha, ha, joking, ì, pay, first, lar, da, stock, comin, aft, finish, lunch, go, str, lor, ard, 3, smth, lor, u, finish, ur, lunch, already, ffffffffff, alright

In [12]:
#find the sum of inidicator function
def sum_indicator(A, w, d):
    v = 0
    for i in range(len(d)):
        temp = -d[i]*A[i].T if (d[i]*A[i]@w < 1) else 0
        v+=temp
        
    return v.reshape(A.shape[1], -1)

# implement a gradient descent method to approach W, look at video 5.7 for formula
def sub_gradientdescent(A,d,tau,lam, w_init,it, tol):
    W = np.zeros((w_init.shape[0], it+1)) #store all steps
    W[:,[0]] = w_init
    for k in range(it):
        W[:, [k+1]] = W[:, [k]] -  tau * (sum_indicator(A,  W[:, [k]], d) + 2*lam*W[:, [k]]) #answer
        if( abs(np.linalg.norm(A@W[:, [k+1]]-d)) < tol):#if meet the tol
            return W[:, [k+1]]
            
    return W[:,[it]]

#implementation begins
def my_svm(A_train, d_train, lamda):
    w_init = np.zeros((A_train.shape[1], 1))
    tol = 10**(-3)
    max_iter = 100
    tau = 0.01
    w_best = sub_gradientdescent(A_train, d_train, tau, lamda, w_init, max_iter, tol)
    d_opt = A_train@w_best
    d_opt = d_opt.reshape(len(d_opt), 1)
    err_cnt = len(d_train) - sum(np.sign(d_opt) == d_train)
    print('best W is: \n', w_best, '\n with error count: ', err_cnt)
    
    return w_best

In [7]:
A = email_data
d = email_class
N = len(email_class)

In [17]:
lamda_arr = np.logspace(1, -3, num = 5)
#choose lamda
A_choose_lam = A[0:500, :]
d_choose_lam = d[0:500, :]
w_init = np.zeros((A_choose_lam.shape[1], 1))
tol = 10**(-3)
max_iter = 100
tau = 0.01
best_lam_index = 0
least_err_cnt = len(d_choose_lam)
w_opt =  np.zeros((A_choose_lam.shape[1], 1))
for j in range(len(lamda_arr)):
        w_best = sub_gradientdescent(A_choose_lam, d_choose_lam, tau, lamda_arr[j], w_init, max_iter, tol)
        d_opt = A_choose_lam@w_best
        d_opt = d_opt.reshape(len(d_opt), 1)
        err_cnt = len(d_choose_lam) - sum(np.sign(d_opt) == d_choose_lam)
        if(err_cnt < least_err_cnt):
            best_lam_index = j
            w_opt = w_best

print('best lamda:', lamda_arr[best_lam_index], '\n')
#use it in later training

best lamda: 0.001 



In [8]:
#each as a validation set for once, other as training
fold = [[1,550],[551,1100],[1101,1650],[1651,2200],[2201,2750],[3301,3850],[4401,4950],[4951,5500],[5501,5571]]
lamda = 0.001 
w_opt = np.zeros((9431, 1))
#abandon last 70 small set
fold.pop(len(fold)-1)
least_err_cnt = 550;
for i in range (len(fold) - 1): 
    vali_ind = np.arange(fold[i][0]-1, fold[i][1])
    training_ind = list(set(range(5500))-set(vali_ind))
    A_train = A[training_ind, :]
    d_train = d[training_ind, 0]
    A_vali =  A[vali_ind, :]
    d_vali = d[vali_ind, 0]
    w_best = np.zeros((9431, 1))
    #train with 
    print('loop', i+1)
    w_best = my_svm(A_train, d_train.reshape(d_train.shape[0], 1), lamda)
    d_test = A_vali@w_best
    vali_err = len(d_vali) - sum(np.sign(d_test).ravel() == d_vali.ravel())
    print('In validation step, error count in 550 data test:', vali_err)
    if(vali_err < least_err_cnt):
        w_opt = w_best
        least_err_cnt = vali_err

print('BEST W: ', w_opt)
print('LEAST ERROR IN 550 DATA:', least_err_cnt)

loop 1
best W is: 
 [[-0.11980677]
 [-0.07984415]
 [-0.00998022]
 ..., 
 [ 0.26949227]
 [ 0.82840832]
 [ 0.00998022]] 
 with error count:  [6]
In validation step, error count in 550 data test: 17
loop 2
best W is: 
 [[-0.07986232]
 [-0.07984415]
 [-0.00998022]
 ..., 
 [ 0.26949227]
 [ 0.80844828]
 [ 0.00998022]] 
 with error count:  [5]
In validation step, error count in 550 data test: 23
loop 3
best W is: 
 [[-0.07986033]
 [-0.07984415]
 [-0.00998022]
 ..., 
 [ 0.26949227]
 [ 0.77849983]
 [ 0.00998022]] 
 with error count:  [6]
In validation step, error count in 550 data test: 28
loop 4
best W is: 
 [[-0.07987911]
 [-0.07984415]
 [ 0.        ]
 ..., 
 [ 0.29943892]
 [ 0.75854977]
 [ 0.00998022]] 
 with error count:  [6]
In validation step, error count in 550 data test: 14
loop 5
best W is: 
 [[-0.09983434]
 [-0.04990309]
 [-0.00998022]
 ..., 
 [ 0.29943892]
 [ 0.76851163]
 [ 0.00998022]] 
 with error count:  [6]
In validation step, error count in 550 data test: 22
loop 6
best W is: 
 

In [9]:
#kernel method using sklearn
import sklearn.svm as svm
#try with ? mark value to see effect

#linear kernel
# C is the penalty parameter of the error term. It controls the trade off
#    between smooth decision boundary and classifying the training points correctly.
advanced_svm = svm.SVC(C = 10, kernel = 'linear')
advanced_svm.fit(A[0:1000], d[0:1000].ravel())
advanced_svm.score(A[1000:2000], d[1000:2000])

0.95599999999999996

In [10]:
# Gaussian/RBF kernel
# gamma is a parameter for non linear hyperplanes. The higher the gamma value it tries to exactly fit the training data set
# C is the penalty parameter of the error term. It controls the trade off
#    between smooth decision boundary and classifying the training points correctly.
advanced_svm = svm.SVC(C = 1, kernel = 'rbf', gamma = 0.1)
advanced_svm.fit(A[0:1000], d[0:1000].ravel())
advanced_svm.score(A[1000:2000], d[1000:2000])

0.92700000000000005

In [11]:
#polynominal kernel
# gamma is a parameter for non linear hyperplanes. The higher the gamma value it tries to exactly fit the training data set
# C is the penalty parameter of the error term. It controls the trade off
#    between smooth decision boundary and classifying the training points correctly.
# degree is a parameter used when kernel is set to ‘poly’.
#    It’s basically the degree of the polynomial used to find the hyperplane to split the data.
advanced_svm = svm.SVC(C = 0.1, kernel = 'poly', gamma = 1, degree = 2)
advanced_svm.fit(A[0:1000], d[0:1000].ravel())
advanced_svm.score(A[1000:2000], d[1000:2000])

0.91400000000000003